<a href="https://colab.research.google.com/github/murarugeorgec/Configurations/blob/master/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Laborator 2: Automate finite deterministe
##Curs: Limbaje formale si automate
##Autor: Pavel Cristian si Muraru George

# Reprezentarea unui automat finit determinist (DFA)
In ceea ce urmeaza va vom prezenta un mod de a reprezenta un ```DFA``` folosind dictionarele din ```python```.

In [0]:
# Setup phase
!pip install -q graphviz

In [0]:
from itertools import product
from graphviz import Digraph, Source


# Vom codifca un DFA ca un dictionar
# Urmatoarele variabile vor fi folosite drept chei in dictionar

ALPHABET = "ALPHABET"
STATES = "STATES"
DELTA = "DELTA"
START_STATE = "START_STATE"
FINAL_STATES = "FINAL_STATES"
SINK_STATE = "SINK"

In [0]:
#@title Functii utilizate pentru graf (nu este nevoie sa modificati)
def to_graph(dfa):
        def get_edges(delta):
            edges = {}
            for (prev_state, symbol), next_state in delta.items():
                edge = (prev_state, next_state)
                if edge not in edges:
                    edges[edge] = set()

                edges[edge].add(symbol)

            return edges

          
        def collate_symbols(dfa, edge_symbols):
            collated = []
            alphabet = dfa[ALPHABET]
            i = 0
            while i < len(alphabet):
                if alphabet[i] not in edge_symbols:
                    i += 1
                    continue

                range_start = i
                while i + 1 < len(dfa[ALPHABET]) and \
                      alphabet[i + 1] in edge_symbols:
                    i += 1

                dist = i - range_start
                if dist >= 2:
                    label = "{}-{}".format(alphabet[range_start], alphabet[i])
                    collated.append(label)
                else:
                    collated.append(str(alphabet[range_start]))
                    if dist == 1:
                        collated.append(str(alphabet[range_start + 1]))
                        i += 1

                i += 1

            return ",".join(collated)

        dot = Digraph()

        # This is here to nicely mark the starting state.
        dot.node("_", shape="point")
        dot.edge("_", str(dfa[START_STATE]))

        for state in dfa[STATES]:
            shape = "doublecircle" if state in dfa[FINAL_STATES] else "circle"
            dot.node(str(state), shape=shape)

        edges = get_edges(dfa[DELTA])

        edges = {k: collate_symbols(dfa, v) for k, v in edges.items()}
        for (prev_state, next_state), label in edges.items():
            dot.edge(str(prev_state), str(next_state), label=label)

        return dot

In [0]:
# Va fi folosita ca sink state
INVALID_STATE = -1


# Sa presupunem ca dorim sa construim automatul care accepta "aaabb"
dfa = {
    ALPHABET: ["a", "b"],
    START_STATE: 0,
    DELTA: {
        (0, "a"): 1,
        (1, "a"): 2,
        (2, "a"): 3,
        (3, "b"): 4,
        (4, "b"): 5
    },
    
    STATES: [0, 1, 2, 3, 4, 5, INVALID_STATE],
    FINAL_STATES: [5],
    SINK_STATE: INVALID_STATE
}
    
# Acum va trebui sa adaugam tranzitii pentru toate celelalte combinatii de
# (stare, simbol) care nu se gasesc deja in lista de tranzitii
all_transitions = (product(dfa[STATES], dfa[ALPHABET]))
not_present_trans = filter(lambda x: x not in dfa[DELTA], all_transitions)
for trans in not_present_trans:
  dfa[DELTA][trans] = dfa[SINK_STATE]
  
print(dfa)
graph = to_graph(dfa)
graph

# Exercitii

##Exercitiul 1:


Scrieti 2 DFA-uri.
* Primul DFA va ccepta string-ul "aaabbbc".
* Al doilea DFA va accepta string-ul "aca".

## Exercitiul 2:

Creati un nou DFA, folosind cele 2 automate definite mai sus, care accepta string-ul "aaabbbcaca" (cele 2 string-uri de mai sus, concatenate).

## Exercitiul 3:

Scrieti un DFA care verifica validitatea unui CNP.

## Exercitiul 4:

Scrieti un DFA care accepta doar date in formatul ZZ/LL/AAAA sau LL/ZZ/AAAA.

## Exercitiul 5:

Scrieti un DFA care verifica daca o adresa IP este valida.

## Exercitiul 6:

Moromete se plimba printr-un labirint modelat ca un grid.

Celulele labirintului pot fi:
* zone libere (Moromete poate pasi in acea zona)
* capcane (Moromete ramane permanent blocat in acea zona);
* iesiri din labirint (Moromete vrea sa ajunga la acestea pentru a scapa din labirint)

Dandu-se un input ca in exemplul de mai jos, codificati si verificati daca simbolurile citite de la intrare
il pot duce pe Moromete spre iesire.
Cand Moromete incearca sa intre intr-un perete al gridului, acesta va ramane pe aceeasi pozitie.

Ex input: "LRSDLRRRU"
Unde:
* D - mutare in jos
* U - mutare in sus
* L - mutare in stanga
* R - mutare in dreapta


In [0]:
# Labirintul codificat
GRID = [
  [0, 0, 0, 0, 0],
  [0, 0, 2, 0, 3],
  [3, 3, 0, 0, 3],
  [0, 0, 0, 0, 0],
  [0, 0, 3, 0, 0],
  [0, 2, 3, 0, 0],
  [0, 0, 0, 1, 0]
]

# Unde
# 0 - celule libere
# 1 - pozitia initiala a lui Moromete
# 2 - celule unde se afla o iesire
# 3 - celule capcana

GOOD_INPUT1 = "LLU"
GOOD_INPUT1 = "RRURULUUUL" # Verifica coliziune cu perete grid
BAD_INPUT1 = "UUL"
BAD_INPUT2 = "DDUL"
BAD_INPUT3 = "LL" # Inputul se termina si nu se ajunge la iesire

## Exercitiul 7:

Scrieti un automat care detecteaza daca un text este spam.

Se va primi:
* Lista de cuvinte care apar de obicei in spam
* Trebuie sa se faca scrierea pentru fiecare cuvant din lista de spam
* Verificare daca un cuvant din text (care va fi split-uit) face parte din lista de spam
* Euristica folosita pentru detectie de spam: $\frac{nr\_cuvinte\_spam}{nr\_cuvinte\_totale}$

In [0]:
spam_words = ['now!','buy!', 'deal']
alphabet = string.ascii_lowercase + ' ,.!?$'


def get_spam_coefficient(input_txt):
  #TODO
  pass



print(get_spam_coefficient("Now! buy! text"))
print(get_spam_coefficient("Hello there good man. Buy! this"))